# GoogleNet 구조

In [2]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt

In [3]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28,28,1))
test_images = test_images.reshape((10000, 28,28,1))

train_images, test_images = train_images / 255.0, test_images / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [18]:
def create_googlenet():
    # creates GoogLeNet a.k.a. Inception v1 (Szegedy, 2015)
    input = tf.keras.Input(shape=(224, 224, 3))

    conv1_7x7_s2 = layers.Conv2D(64, (7,7), strides=(2,2), padding='same', activation='relu', 
                          name='conv1/7x7_s2')(input)
    pool1_3x3_s2 = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', 
                                       name='pool1/3x3_s2')(conv1_7x7_s2)
    pool1_norm1 = layers.Lambda(tf.nn.local_response_normalization, name='pool1/norm1')(pool1_3x3_s2)

    conv2_3x3_reduce = layers.Conv2D(64, (1,1), padding='valid', 
                            activation='relu', name='conv2/3x3_reduce')(pool1_norm1)
    conv2_3x3 = layers.Conv2D(192, (3,3), padding='same', 
                            activation='relu', name='conv2/3x3')(conv2_3x3_reduce)
    conv2_norm2 = layers.Lambda(tf.nn.local_response_normalization, name='conv2/norm2')(conv2_3x3)
    pool2_3x3_s2 = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', 
                                       name='pool2/3x3_s2')(conv2_norm2)

    # Inception 3a ############################################################
    inception_3a_1x1 = layers.Conv2D(64, (1,1), padding='same', activation='relu', 
                              name='inception_3a/1x1')(pool2_3x3_s2)

    inception_3a_3x3_reduce = layers.Conv2D(96, (1,1), padding='same', activation='relu', 
                                     name='inception_3a/3x3_reduce')(pool2_3x3_s2)
    inception_3a_3x3 = layers.Conv2D(128, (3,3), padding='same', activation='relu', 
                              name='inception_3a/3x3')(inception_3a_3x3_reduce)

    inception_3a_5x5_reduce = layers.Conv2D(16, (1,1), padding='same', activation='relu',
                                     name='inception_3a/5x5_reduce')(pool2_3x3_s2)
    inception_3a_5x5 = layers.Conv2D(32, (5,5), padding='same', activation='relu', 
                              name='inception_3a/5x5')(inception_3a_5x5_reduce)
    
    inception_3a_pool = layers.MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                     name='inception_3a/pool')(pool2_3x3_s2)
    inception_3a_pool_proj = layers.Conv2D(32, (1,1), padding='same', activation='relu', 
                                    name='inception_3a/pool_proj')(inception_3a_pool)
    # output : concat
    inception_3a_output = layers.Concatenate(axis=-1, 
                                      name='inception_3a/output')([inception_3a_1x1,inception_3a_3x3,
                                                                   inception_3a_5x5,inception_3a_pool_proj])

    # Inception 3b ############################################################
    inception_3b_1x1 = layers.Conv2D(128, (1,1), padding='same', activation='relu', 
                              name='inception_3b/1x1')(inception_3a_output)

    inception_3b_3x3_reduce = layers.Conv2D(128, (1,1), padding='same', activation='relu',
                                     name='inception_3b/3x3_reduce')(inception_3a_output)
    inception_3b_3x3 = layers.Conv2D(192, (3,3), padding='same', activation='relu', 
                              name='inception_3b/3x3')(inception_3b_3x3_reduce)
    inception_3b_5x5_reduce = layers.Conv2D(32, (1,1), padding='same', activation='relu', 
                                     name='inception_3b/5x5_reduce')(inception_3a_output)

    inception_3b_5x5 = layers.Conv2D(96, (5,5), padding='same', activation='relu', 
                              name='inception_3b/5x5')(inception_3b_5x5_reduce)

    inception_3b_pool = layers.MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                     name='inception_3b/pool')(inception_3a_output)

    inception_3b_pool_proj = layers.Conv2D(64, (1,1), padding='same', activation='relu', 
                                    name='inception_3b/pool_proj')(inception_3b_pool)
    # output : concat
    inception_3b_output = layers.Concatenate(axis=-1, 
                                      name='inception_3b/output')([inception_3b_1x1,inception_3b_3x3,
                                                                   inception_3b_5x5,inception_3b_pool_proj])

    pool3_3x3_s2 = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', name='pool3/3x3_s2')(inception_3b_output)


    ###########################################################################
    # Inception 4a ############################################################
    inception_4a_1x1 = layers.Conv2D(192, (1,1), padding='same', activation='relu', 
                              name='inception_4a/1x1')(pool3_3x3_s2)

    inception_4a_3x3_reduce = layers.Conv2D(96, (1,1), padding='same', activation='relu', 
                                     name='inception_4a/3x3_reduce')(pool3_3x3_s2)
    inception_4a_3x3 = layers.Conv2D(208, (3,3), padding='same', activation='relu', 
                              name='inception_4a/3x3')(inception_4a_3x3_reduce)

    inception_4a_5x5_reduce = layers.Conv2D(16, (1,1), padding='same', activation='relu', 
                                     name='inception_4a/5x5_reduce')(pool3_3x3_s2)
    inception_4a_5x5 = layers.Conv2D(48, (5,5), padding='same', activation='relu', 
                              name='inception_4a/5x5')(inception_4a_5x5_reduce)

    inception_4a_pool = layers.MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                     name='inception_4a/pool')(pool3_3x3_s2)
    inception_4a_pool_proj = layers.Conv2D(64, (1,1), padding='same', activation='relu', 
                                    name='inception_4a/pool_proj')(inception_4a_pool)
    
    # output : concat
    inception_4a_output = layers.Concatenate(axis=-1, 
                                      name='inception_4a/output')([inception_4a_1x1,inception_4a_3x3,
                                                                   inception_4a_5x5,inception_4a_pool_proj])

    # auxiliary classifier **********************
    loss1_avg_pool = layers.AveragePooling2D(pool_size=(5,5), strides=(3,3), 
                                             name='loss1/avg_pool')(inception_4a_output)
    loss1_conv = layers.Conv2D(128, (1,1), padding='same', activation='relu', 
                               name='loss1/conv')(loss1_avg_pool)
    loss1_flat = layers.Flatten()(loss1_conv)
    loss1_fc = layers.Dense(1024, activation='relu', name='loss1/fc')(loss1_flat)
    loss1_drop_fc = layers.Dropout(rate=0.7)(loss1_fc)
    loss1_classifier = layers.Dense(1000, activation='softmax', name='loss1/classifier')(loss1_drop_fc)

    # Inception 4b ############################################################
    inception_4b_1x1 = layers.Conv2D(160, (1,1), padding='same', activation='relu',
                              name='inception_4b/1x1')(inception_4a_output)

    inception_4b_3x3_reduce = layers.Conv2D(112, (1,1), padding='same', activation='relu', 
                                     name='inception_4b/3x3_reduce')(inception_4a_output)
    inception_4b_3x3 = layers.Conv2D(224, (3,3), padding='same', activation='relu', 
                              name='inception_4b/3x3')(inception_4b_3x3_reduce)

    inception_4b_5x5_reduce = layers.Conv2D(24, (1,1), padding='same', activation='relu', 
                                     name='inception_4b/5x5_reduce')(inception_4a_output)
    inception_4b_5x5 = layers.Conv2D(64, (5,5), padding='same', activation='relu', 
                              name='inception_4b/5x5')(inception_4b_5x5_reduce)

    inception_4b_pool = layers.MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                     name='inception_4b/pool')(inception_4a_output)
    inception_4b_pool_proj = layers.Conv2D(64, (1,1), padding='same', activation='relu', 
                                    name='inception_4b/pool_proj')(inception_4b_pool)
    
    # output : concat
    inception_4b_output = layers.Concatenate(axis=-1, 
                                      name='inception_4b/output')([inception_4b_1x1,inception_4b_3x3,
                                                                   inception_4b_5x5,inception_4b_pool_proj])

    # Inception 4c ############################################################
    inception_4c_1x1 = layers.Conv2D(128, (1,1), padding='same', activation='relu', 
                              name='inception_4c/1x1')(inception_4b_output)

    inception_4c_3x3_reduce = layers.Conv2D(128, (1,1), padding='same', activation='relu', 
                                     name='inception_4c/3x3_reduce')(inception_4b_output)
    inception_4c_3x3 = layers.Conv2D(256, (3,3), padding='same', activation='relu', 
                              name='inception_4c/3x3')(inception_4c_3x3_reduce)

    inception_4c_5x5_reduce = layers.Conv2D(24, (1,1), padding='same', activation='relu', 
                                     name='inception_4c/5x5_reduce')(inception_4b_output)
    inception_4c_5x5 = layers.Conv2D(64, (5,5), padding='same', activation='relu', 
                              name='inception_4c/5x5')(inception_4c_5x5_reduce)

    inception_4c_pool = layers.MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                                        name='inception_4c/pool')(inception_4b_output)
    inception_4c_pool_proj = layers.Conv2D(64, (1,1), padding='same', activation='relu', 
                                    name='inception_4c/pool_proj')(inception_4c_pool)

    # output : concat
    inception_4c_output = layers.Concatenate(axis=-1, 
                                      name='inception_4c/output')([inception_4c_1x1,inception_4c_3x3,
                                                                   inception_4c_5x5,inception_4c_pool_proj])

    # Inception 4d ############################################################
    inception_4d_1x1 = layers.Conv2D(112, (1,1), padding='same', activation='relu', 
                              name='inception_4d/1x1')(inception_4c_output)

    inception_4d_3x3_reduce = layers.Conv2D(144, (1,1), padding='same', activation='relu',
                                     name='inception_4d/3x3_reduce')(inception_4c_output)
    inception_4d_3x3 = layers.Conv2D(288, (3,3), padding='same', activation='relu', 
                              name='inception_4d/3x3')(inception_4d_3x3_reduce)

    inception_4d_5x5_reduce = layers.Conv2D(32, (1,1), padding='same', activation='relu', 
                                     name='inception_4d/5x5_reduce')(inception_4c_output)
    inception_4d_5x5 = layers.Conv2D(64, (5,5), padding='same', activation='relu', 
                              name='inception_4d/5x5')(inception_4d_5x5_reduce)

    inception_4d_pool = layers.MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                                        name='inception_4d/pool')(inception_4c_output)
    inception_4d_pool_proj = layers.Conv2D(64, (1,1), padding='same', activation='relu', 
                                    name='inception_4d/pool_proj')(inception_4d_pool)

    # output : concat
    inception_4d_output = layers.Concatenate(axis=-1, 
                                      name='inception_4d/output')([inception_4d_1x1,inception_4d_3x3,
                                                                   inception_4d_5x5,inception_4d_pool_proj])

    # auxiliary classifier **********************
    loss2_avg_pool = layers.AveragePooling2D(pool_size=(5,5), strides=(3,3), 
                                             name='loss2/avg_pool')(inception_4d_output)
    loss2_conv = layers.Conv2D(128, (1,1), padding='same', activation='relu', 
                               name='loss2/conv')(loss2_avg_pool)
    loss2_flat = layers.Flatten()(loss2_conv)
    loss2_fc = layers.Dense(1024, activation='relu', name='loss2/fc')(loss2_flat)
    loss2_drop_fc = layers.Dropout(rate=0.7)(loss2_fc)
    loss2_classifier = layers.Dense(1000, activation='softmax', name='loss2/classifier')(loss2_drop_fc)

    # Inception 4e ############################################################
    inception_4e_1x1 = layers.Conv2D(256, (1,1), padding='same', activation='relu', 
                              name='inception_4e/1x1')(inception_4d_output)

    inception_4e_3x3_reduce = layers.Conv2D(160, (1,1), padding='same', activation='relu',
                                     name='inception_4e/3x3_reduce')(inception_4d_output)
    inception_4e_3x3 = layers.Conv2D(320, (3,3), padding='same', activation='relu', 
                              name='inception_4e/3x3')(inception_4e_3x3_reduce)

    inception_4e_5x5_reduce = layers.Conv2D(32, (1,1), padding='same', activation='relu', 
                                     name='inception_4e/5x5_reduce')(inception_4d_output)
    inception_4e_5x5 = layers.Conv2D(128, (5,5), padding='same', activation='relu', 
                              name='inception_4e/5x5')(inception_4e_5x5_reduce)

    inception_4e_pool = layers.MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                                        name='inception_4e/pool')(inception_4d_output)
    inception_4e_pool_proj = layers.Conv2D(128, (1,1), padding='same', activation='relu', 
                                    name='inception_4e/pool_proj')(inception_4e_pool)

    # output : concat
    inception_4e_output = layers.Concatenate(axis=-1, 
                                      name='inception_4e/output')([inception_4e_1x1,inception_4e_3x3,
                                                                   inception_4e_5x5,inception_4e_pool_proj])

    pool4_3x3_s2 = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', 
                                       name='pool4/3x3_s2')(inception_4e_output)

    ###########################################################################
    # Inception 5a ############################################################
    inception_5a_1x1 = layers.Conv2D(256, (1,1), padding='same', activation='relu', 
                              name='inception_5a/1x1')(pool4_3x3_s2)

    inception_5a_3x3_reduce = layers.Conv2D(160, (1,1), padding='same', activation='relu', 
                                     name='inception_5a/3x3_reduce')(pool4_3x3_s2)
    inception_5a_3x3 = layers.Conv2D(320, (3,3), padding='same', activation='relu', 
                              name='inception_5a/3x3')(inception_5a_3x3_reduce)

    inception_5a_5x5_reduce = layers.Conv2D(32, (1,1), padding='same', activation='relu', 
                                     name='inception_5a/5x5_reduce')(pool4_3x3_s2)
    inception_5a_5x5 = layers.Conv2D(128, (5,5), padding='same', activation='relu', 
                              name='inception_5a/5x5')(inception_5a_5x5_reduce)

    inception_5a_pool = layers.MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                                        name='inception_5a/pool')(pool4_3x3_s2)
    inception_5a_pool_proj = layers.Conv2D(128, (1,1), padding='same', activation='relu', 
                                    name='inception_5a/pool_proj')(inception_5a_pool)

    # output : concat
    inception_5a_output = layers.Concatenate(axis=-1,
                                      name='inception_5a/output')([inception_5a_1x1,inception_5a_3x3,
                                                                   inception_5a_5x5,inception_5a_pool_proj])

    # Inception 5b ############################################################
    inception_5b_1x1 = layers.Conv2D(384, (1,1), padding='same', activation='relu', 
                              name='inception_5b/1x1')(inception_5a_output)

    inception_5b_3x3_reduce = layers.Conv2D(192, (1,1), padding='same', activation='relu', 
                                     name='inception_5b/3x3_reduce')(inception_5a_output)
    inception_5b_3x3 = layers.Conv2D(384, (3,3), padding='same', activation='relu', 
                              name='inception_5b/3x3')(inception_5b_3x3_reduce)

    inception_5b_5x5_reduce = layers.Conv2D(48, (1,1), padding='same', activation='relu', 
                                     name='inception_5b/5x5_reduce')(inception_5a_output)
    inception_5b_5x5 = layers.Conv2D(128, (5,5), padding='same', activation='relu', 
                              name='inception_5b/5x5')(inception_5b_5x5_reduce)

    inception_5b_pool = layers.MaxPooling2D(pool_size=(3,3), strides=(1,1), padding='same', 
                                                        name='inception_5b/pool')(inception_5a_output)
    inception_5b_pool_proj = layers.Conv2D(128, (1,1), padding='same', activation='relu', 
                                    name='inception_5b/pool_proj')(inception_5b_pool)

    # output : concat
    inception_5b_output = layers.Concatenate(axis=-1, 
                                      name='inception_5b/output')([inception_5b_1x1,inception_5b_3x3,
                                                                   inception_5b_5x5,inception_5b_pool_proj])

    pool5_7x7_s1 = layers.AveragePooling2D(pool_size=(7,7), strides=(1,1), name='pool5/7x7_s2')(inception_5b_output)
    loss3_flat = layers.Flatten()(pool5_7x7_s1)
    pool5_drop_7x7_s1 = layers.Dropout(rate=0.4)(loss3_flat)
    loss3_classifier = layers.Dense(1000, activation='softmax', name='loss3/classifier')(pool5_drop_7x7_s1)


   # Final Model : input + output[aux.cls1, aux.cls2, main.cls]
    googlenet = tf.keras.Model(inputs=input, outputs=[loss1_classifier,loss2_classifier,loss3_classifier])

    return googlenet

In [19]:
model = create_googlenet()

In [20]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1/7x7_s2 (Conv2D)          (None, 112, 112, 64  9472        ['input_8[0][0]']                
                                )                                                                 
                                                                                                  
 pool1/3x3_s2 (MaxPooling2D)    (None, 56, 56, 64)   0           ['conv1/7x7_s2[0][0]']           
                                                                                            